AI For Social Good Club - Sign Language Project

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'tensorflow'